In [ ]:
#labelme got isolating target in image
!pip install labelme  tensorflow opencv-python matplotlib albumentations

In [1]:
import sys
print(sys.executable)

C:\Users\aalsa\development\img_rec_proj\face_detection\Scripts\python.exe


In [2]:
#!pip install labelme  tensorflow opencv-python

In [3]:
import os
import time
import uuid #creates unique identifier, for filenames
import cv2 #works with files and cameras/ audio to allow computer vison

In [4]:
IMAGES_PATH= os.path.join('data','images')
number_images=20

In [5]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
print("you")

you
